# Yolov8 Model Training

This notebook helps easily train a yolo model. Model evaluation will happen in another notebook.

## 1. Imports and variable setup

In [1]:
# Import all libraries
import os
from roboflow import Roboflow
from IPython import display
import ultralytics
from ultralytics import YOLO
from IPython.display import display, Image
from pathlib import Path
from dotenv import find_dotenv, load_dotenv
import sys
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Setup HOME environment variable
HOME = os.getcwd()
print(HOME)

/home/mreag/repos/DBD-Killer-AI/notebooks


In [2]:
# Check image displays are good
# display.clear_output()
sys.path.append(str(Path.cwd().parent))
# Check ultralytics library is good
ultralytics.checks()

Ultralytics YOLOv8.0.227 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (NVIDIA GeForce RTX 3080, 10240MiB)
Setup complete ✅ (12 CPUs, 7.7 GB RAM, 68.7/1006.9 GB disk)


In [3]:
# Environment variables
PROJECT_DIR = Path.cwd().parent
DATASET_VERSION = 5

## 2. Import Model

Pre-trained is the ideal model

In [4]:
model = YOLO(model=str(PROJECT_DIR) + "/models/yolov8n.pt")

## 3. Train Model

In [5]:
from dbdkillerai.data.make_dataset import roboflow_connect, roboflow_download
from dbdkillerai.models.train_model import train_yolo, validate_yolo

# Establish Roboflow connection and acquire dataset location. DONT download.
rf_conn, rf_project = roboflow_connect()
data_location = roboflow_download(rf_project=rf_project,
                                  rf_data_version=DATASET_VERSION,
                                  data_format="yolov8",
                                  project_dir=PROJECT_DIR,
                                  overwrite=False)

# Get location of yml
yml_location = str(PROJECT_DIR) + data_location.location.replace(
    str(PROJECT_DIR), "/data/external"
) + "/data.yaml"
model, results_train = train_yolo(yolo_model=model,
                                    data_yml=yml_location,
                                    epochs=1,
                                    imgsz=800,
                                    plots=True,
                                    workers=0)



loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.0.227, to fix: `pip install ultralytics==8.0.196`
New https://pypi.org/project/ultralytics/8.1.2 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.227 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (NVIDIA GeForce RTX 3080, 10240MiB)
engine/trainer: task=detect, mode=train, model=/home/mreag/repos/DBD-Killer-AI/models/yolov8n.pt, data=/home/mreag/repos/DBD-Killer-AI/data/external/deadbydaylightkillerai/killer_ai_object_detection/5/data.yaml, epochs=1, patience=50, batch=16, imgsz=800, save=True, save_period=-1, cache=False, device=None, workers=0, project=None, name=train10, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val

train: Scanning /home/mreag/repos/DBD-Killer-AI/data/external/deadbydaylightkillerai/killer_ai_object_detection/5/train/labels.cache... 1468 images, 554 backgrounds, 0 corrupt: 100%|██████████| 1468/1468 [00:00<?, ?it/s]
val: Scanning /home/mreag/repos/DBD-Killer-AI/data/external/deadbydaylightkillerai/killer_ai_object_detection/5/valid/labels.cache... 238 images, 93 backgrounds, 0 corrupt: 100%|██████████| 238/238 [00:00<?, ?it/s]

Plotting labels to runs/detect/train10/labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 800 train, 800 val
Using 0 dataloader workers
Logging results to runs/detect/train10
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1      3.27G      1.523       3.85      1.271         24        800: 100%|██████████| 92/92 [00:33<00:00,  2.78it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:04<00:00,  1.77it/s]


                   all        238        309      0.586      0.185      0.287      0.174

1 epochs completed in 0.011 hours.
Optimizer stripped from runs/detect/train10/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train10/weights/best.pt, 6.2MB

Validating runs/detect/train10/weights/best.pt...
Ultralytics YOLOv8.0.227 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (NVIDIA GeForce RTX 3080, 10240MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:03<00:00,  2.32it/s]


                   all        238        309      0.585      0.185      0.288      0.174
             generator        238        189       0.95      0.203      0.478      0.293
                  hook        238        120       0.22      0.167     0.0979      0.056
Speed: 0.2ms preprocess, 1.9ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs/detect/train10


## Validate Custom Model

In [6]:
best_weights_location = str(results_train.save_dir) + "/weights/best.pt"
best_model = YOLO(best_weights_location)
results_val = validate_yolo(yolo_model=best_model,
                            data_yml=yml_location)


Ultralytics YOLOv8.0.227 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (NVIDIA GeForce RTX 3080, 10240MiB)


Model summary (fused): 168 layers, 3006038 parameters, 0 gradients


val: Scanning /home/mreag/repos/DBD-Killer-AI/data/external/deadbydaylightkillerai/killer_ai_object_detection/5/valid/labels.cache... 238 images, 93 backgrounds, 0 corrupt: 100%|██████████| 238/238 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:02<00:00,  6.51it/s]


                   all        238        309      0.586      0.185      0.288      0.174
             generator        238        189       0.95      0.203      0.477      0.292
                  hook        238        120      0.221      0.167     0.0979     0.0558
Speed: 0.2ms preprocess, 2.8ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to runs/detect/val2


: 